# Tarefa 4

<b>Alunos:</b> Andreza (164213), Gil (225323) e Yan (118982)

In [1]:
import time
import numpy as np
from scipy.stats import loguniform, uniform

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score

from hyperopt import tpe, fmin, hp, STATUS_OK
from pyswarm import pso
from simanneal import Annealer
import optuna

## Hiperparâmetros:
* C: 2^{-5} a 2^{15} (uniforme nos expoentes)
* gamma: 2^{-15} a 2^{3} (uniforme nos expoentes)
* epsilon: 0.05 a 1.0 (uniforme no intervalo)

In [2]:
# Defines
exp_min_C = -5
exp_max_C = 15
exp_min_gamma = -15
exp_max_gamma = 3
min_epsilon = 0.05
max_epsilon = 1.0

## Carregamento dos dados

In [3]:
X = np.load("X.npy")
y = np.load("y.npy")

## Random Search

In [4]:
param_distributions = {'C': loguniform(2**exp_min_C, 2**exp_max_C), 
                       'gamma': loguniform(2**exp_min_gamma, 2**exp_max_gamma) ,
                       'epsilon': uniform(min_epsilon, max_epsilon)}

start = time.time()
# Realiza a busca aleatória
# Se cv = None, utiliza 5 fold cross validation
random_search = RandomizedSearchCV(
    SVR(),
    param_distributions,
    random_state = 0,
    n_iter=125,
    scoring = 'neg_mean_squared_error',
    cv = None,
    refit = True,
    verbose=2)
random_search.fit(X, y)
time_elapsed = time.time() - start

Fitting 5 folds for each of 125 candidates, totalling 625 fits
[CV] END C=62.955917245645935, epsilon=0.7651893663724195, gamma=0.05631803349608218; total time=   0.0s
[CV] END C=62.955917245645935, epsilon=0.7651893663724195, gamma=0.05631803349608218; total time=   0.0s
[CV] END C=62.955917245645935, epsilon=0.7651893663724195, gamma=0.05631803349608218; total time=   0.0s
[CV] END C=62.955917245645935, epsilon=0.7651893663724195, gamma=0.05631803349608218; total time=   0.0s
[CV] END C=62.955917245645935, epsilon=0.7651893663724195, gamma=0.05631803349608218; total time=   0.0s
[CV] END C=59.617487041755574, epsilon=0.4736547993389047, gamma=0.09646129810622778; total time=   0.0s
[CV] END C=59.617487041755574, epsilon=0.4736547993389047, gamma=0.09646129810622778; total time=   0.0s
[CV] END C=59.617487041755574, epsilon=0.4736547993389047, gamma=0.09646129810622778; total time=   0.0s
[CV] END C=59.617487041755574, epsilon=0.4736547993389047, gamma=0.09646129810622778; total time=

[CV] END C=2.476953900804805, epsilon=0.4137107709426226, gamma=0.03751319278155217; total time=   0.0s
[CV] END C=2.476953900804805, epsilon=0.4137107709426226, gamma=0.03751319278155217; total time=   0.0s
[CV] END C=2.476953900804805, epsilon=0.4137107709426226, gamma=0.03751319278155217; total time=   0.0s
[CV] END C=13.66137005704298, epsilon=1.0383738380592262, gamma=0.00010901441934611322; total time=   0.0s
[CV] END C=13.66137005704298, epsilon=1.0383738380592262, gamma=0.00010901441934611322; total time=   0.0s
[CV] END C=13.66137005704298, epsilon=1.0383738380592262, gamma=0.00010901441934611322; total time=   0.0s
[CV] END C=13.66137005704298, epsilon=1.0383738380592262, gamma=0.00010901441934611322; total time=   0.0s
[CV] END C=13.66137005704298, epsilon=1.0383738380592262, gamma=0.00010901441934611322; total time=   0.0s
[CV] END C=0.5654749896524898, epsilon=0.21130951788499625, gamma=0.10554644254225594; total time=   0.0s
[CV] END C=0.5654749896524898, epsilon=0.211309

[CV] END C=833.986628433057, epsilon=1.0121885451174382, gamma=0.0006798747371434141; total time=   0.1s
[CV] END C=833.986628433057, epsilon=1.0121885451174382, gamma=0.0006798747371434141; total time=   0.1s
[CV] END C=833.986628433057, epsilon=1.0121885451174382, gamma=0.0006798747371434141; total time=   0.1s
[CV] END C=833.986628433057, epsilon=1.0121885451174382, gamma=0.0006798747371434141; total time=   0.1s
[CV] END C=833.986628433057, epsilon=1.0121885451174382, gamma=0.0006798747371434141; total time=   0.1s
[CV] END C=91.97352238038614, epsilon=0.6420419312718391, gamma=0.038487514185507; total time=   0.0s
[CV] END C=91.97352238038614, epsilon=0.6420419312718391, gamma=0.038487514185507; total time=   0.0s
[CV] END C=91.97352238038614, epsilon=0.6420419312718391, gamma=0.038487514185507; total time=   0.0s
[CV] END C=91.97352238038614, epsilon=0.6420419312718391, gamma=0.038487514185507; total time=   0.0s
[CV] END C=91.97352238038614, epsilon=0.6420419312718391, gamma=0.0

[CV] END C=2266.702244003744, epsilon=0.6191007386145934, gamma=0.004907824413128085; total time=   0.0s
[CV] END C=2266.702244003744, epsilon=0.6191007386145934, gamma=0.004907824413128085; total time=   0.0s
[CV] END C=2266.702244003744, epsilon=0.6191007386145934, gamma=0.004907824413128085; total time=   0.0s
[CV] END C=2266.702244003744, epsilon=0.6191007386145934, gamma=0.004907824413128085; total time=   0.0s
[CV] END C=2266.702244003744, epsilon=0.6191007386145934, gamma=0.004907824413128085; total time=   0.0s
[CV] END C=0.08152237593653709, epsilon=0.7474287731445637, gamma=0.008751635019606737; total time=   0.0s
[CV] END C=0.08152237593653709, epsilon=0.7474287731445637, gamma=0.008751635019606737; total time=   0.0s
[CV] END C=0.08152237593653709, epsilon=0.7474287731445637, gamma=0.008751635019606737; total time=   0.0s
[CV] END C=0.08152237593653709, epsilon=0.7474287731445637, gamma=0.008751635019606737; total time=   0.0s
[CV] END C=0.08152237593653709, epsilon=0.74742

[CV] END C=16.841361755060674, epsilon=0.5865792111087222, gamma=2.203937442969064; total time=   0.0s
[CV] END C=16.841361755060674, epsilon=0.5865792111087222, gamma=2.203937442969064; total time=   0.0s
[CV] END C=16.841361755060674, epsilon=0.5865792111087222, gamma=2.203937442969064; total time=   0.0s
[CV] END C=28660.555423362377, epsilon=0.2668969843984739, gamma=0.11952693411223328; total time=   0.0s
[CV] END C=28660.555423362377, epsilon=0.2668969843984739, gamma=0.11952693411223328; total time=   0.0s
[CV] END C=28660.555423362377, epsilon=0.2668969843984739, gamma=0.11952693411223328; total time=   0.0s
[CV] END C=28660.555423362377, epsilon=0.2668969843984739, gamma=0.11952693411223328; total time=   0.0s
[CV] END C=28660.555423362377, epsilon=0.2668969843984739, gamma=0.11952693411223328; total time=   0.0s
[CV] END C=1.2028423216739412, epsilon=0.07065099946572868, gamma=0.3925139324355619; total time=   0.0s
[CV] END C=1.2028423216739412, epsilon=0.07065099946572868, g

[CV] END C=19332.32550715436, epsilon=0.34214752679254884, gamma=0.0006158719483657019; total time=   1.5s
[CV] END C=19332.32550715436, epsilon=0.34214752679254884, gamma=0.0006158719483657019; total time=   1.1s
[CV] END C=19332.32550715436, epsilon=0.34214752679254884, gamma=0.0006158719483657019; total time=   2.0s
[CV] END C=19332.32550715436, epsilon=0.34214752679254884, gamma=0.0006158719483657019; total time=   1.0s
[CV] END C=19332.32550715436, epsilon=0.34214752679254884, gamma=0.0006158719483657019; total time=   1.2s
[CV] END C=0.12551040234572877, epsilon=0.0664296295914742, gamma=3.3207927843274354; total time=   0.0s
[CV] END C=0.12551040234572877, epsilon=0.0664296295914742, gamma=3.3207927843274354; total time=   0.0s
[CV] END C=0.12551040234572877, epsilon=0.0664296295914742, gamma=3.3207927843274354; total time=   0.0s
[CV] END C=0.12551040234572877, epsilon=0.0664296295914742, gamma=3.3207927843274354; total time=   0.0s
[CV] END C=0.12551040234572877, epsilon=0.066

[CV] END C=0.0941067261636307, epsilon=0.13960303423860537, gamma=0.1336804622575878; total time=   0.0s
[CV] END C=0.0941067261636307, epsilon=0.13960303423860537, gamma=0.1336804622575878; total time=   0.0s
[CV] END C=0.9377948081229858, epsilon=0.47053946668009844, gamma=0.03196502968441532; total time=   0.0s
[CV] END C=0.9377948081229858, epsilon=0.47053946668009844, gamma=0.03196502968441532; total time=   0.0s
[CV] END C=0.9377948081229858, epsilon=0.47053946668009844, gamma=0.03196502968441532; total time=   0.0s
[CV] END C=0.9377948081229858, epsilon=0.47053946668009844, gamma=0.03196502968441532; total time=   0.0s
[CV] END C=0.9377948081229858, epsilon=0.47053946668009844, gamma=0.03196502968441532; total time=   0.0s
[CV] END C=4741.157079696716, epsilon=0.7770442627113283, gamma=0.0008898823880515364; total time=   0.2s
[CV] END C=4741.157079696716, epsilon=0.7770442627113283, gamma=0.0008898823880515364; total time=   0.2s
[CV] END C=4741.157079696716, epsilon=0.77704426

[CV] END C=17.608845672047725, epsilon=0.9320414102298896, gamma=0.009322103925178255; total time=   0.0s
[CV] END C=17.608845672047725, epsilon=0.9320414102298896, gamma=0.009322103925178255; total time=   0.0s
[CV] END C=17.608845672047725, epsilon=0.9320414102298896, gamma=0.009322103925178255; total time=   0.0s
[CV] END C=715.7702306898558, epsilon=0.449025321703102, gamma=2.4162991392595385; total time=   0.0s
[CV] END C=715.7702306898558, epsilon=0.449025321703102, gamma=2.4162991392595385; total time=   0.0s
[CV] END C=715.7702306898558, epsilon=0.449025321703102, gamma=2.4162991392595385; total time=   0.0s
[CV] END C=715.7702306898558, epsilon=0.449025321703102, gamma=2.4162991392595385; total time=   0.0s
[CV] END C=715.7702306898558, epsilon=0.449025321703102, gamma=2.4162991392595385; total time=   0.0s
[CV] END C=445.8765154196025, epsilon=0.7496220542505168, gamma=0.0018210259897901138; total time=   0.0s
[CV] END C=445.8765154196025, epsilon=0.7496220542505168, gamma=0.

In [5]:
print("Tempo de execução Random Search: {:.3f} s".format(time_elapsed))
print("Melhores parâmetros: ", random_search.best_params_)
regressor = SVR(kernel = 'rbf', C = random_search.best_params_['C'],
                gamma = random_search.best_params_['gamma'],
                epsilon = random_search.best_params_['epsilon'])

scores = cross_val_score(regressor, X, y, scoring = 'neg_mean_squared_error', cv=5)
rmse =  np.sqrt(np.mean(np.absolute(scores)))
print("RMSE:", rmse)

Tempo de execução Random Search: 33.768 s
Melhores parâmetros:  {'C': 12090.831718005036, 'epsilon': 0.7544144019235328, 'gamma': 4.540153912140535e-05}
RMSE: 4.100037405923898


## Grid Search

Fazemos um grid search com grid de 5x5x5 selecionando uniformemente 5 expoentes de C e gamma e 5 valores de epsilon

In [6]:
# C_values = [2**float(expo) for expo in np.random.uniform(low = exp_min_C, high = exp_max_C, size = 5)]
# gamma_values = [2**float(expo) for expo in np.random.uniform(low = exp_min_gamma, high = exp_max_gamma, size = 5)]
param_grid = {'C': loguniform.rvs(2**exp_min_C, 2**exp_max_C, size = 5), 
              'gamma': loguniform.rvs(2**exp_min_gamma, 2**exp_max_gamma,  size = 5) ,
              'epsilon': uniform.rvs(min_epsilon, max_epsilon, size= 5)}
print(param_grid)
strt = time.time()
# Realiza o Grid Search utilizando os parametros definidos em param_grid
grid_search = GridSearchCV(SVR(),
                           param_grid,
                           cv = None,
                           scoring = 'neg_mean_squared_error',
                           refit = True,
                           verbose = 2)
grid_search.fit(X, y)
time_elapsed = time.time() - start

{'C': array([5.60751187e+00, 2.73758429e+02, 1.74381617e+04, 1.60271906e+03,
       1.98915448e+02]), 'gamma': array([0.00028876, 0.03939625, 0.00703171, 0.03064538, 0.00316841]), 'epsilon': array([0.8354723 , 0.37272008, 0.12768522, 0.09569579, 0.24120751])}
Fitting 5 folds for each of 125 candidates, totalling 625 fits
[CV] END C=5.607511871281818, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.8354722980853039, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=

[CV] END C=5.607511871281818, epsilon=0.09569579080821362, gamma=0.007031706915999999; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.09569579080821362, gamma=0.007031706915999999; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.09569579080821362, gamma=0.030645378763400424; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.09569579080821362, gamma=0.030645378763400424; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.09569579080821362, gamma=0.030645378763400424; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.09569579080821362, gamma=0.030645378763400424; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.09569579080821362, gamma=0.030645378763400424; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.09569579080821362, gamma=0.0031684076500385137; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.09569579080821362, gamma=0.0031684076500385137; total time=   0.0s
[CV] END C=5.607511871281818, epsilon=0.0956

[CV] END C=273.7584285973906, epsilon=0.372720081753474, gamma=0.0031684076500385137; total time=   0.0s
[CV] END C=273.7584285973906, epsilon=0.372720081753474, gamma=0.0031684076500385137; total time=   0.0s
[CV] END C=273.7584285973906, epsilon=0.372720081753474, gamma=0.0031684076500385137; total time=   0.0s
[CV] END C=273.7584285973906, epsilon=0.12768522363993035, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=273.7584285973906, epsilon=0.12768522363993035, gamma=0.00028875853933915967; total time=   0.1s
[CV] END C=273.7584285973906, epsilon=0.12768522363993035, gamma=0.00028875853933915967; total time=   0.1s
[CV] END C=273.7584285973906, epsilon=0.12768522363993035, gamma=0.00028875853933915967; total time=   0.1s
[CV] END C=273.7584285973906, epsilon=0.12768522363993035, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=273.7584285973906, epsilon=0.12768522363993035, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=273.7584285973906, epsilon=0.

[CV] END C=17438.16167958451, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.8s
[CV] END C=17438.16167958451, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.5s
[CV] END C=17438.16167958451, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.6s
[CV] END C=17438.16167958451, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.7s
[CV] END C=17438.16167958451, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.7s
[CV] END C=17438.16167958451, epsilon=0.8354722980853039, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=17438.16167958451, epsilon=0.8354722980853039, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=17438.16167958451, epsilon=0.8354722980853039, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=17438.16167958451, epsilon=0.8354722980853039, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=17438.16167958451, epsilon=0.835472298

[CV] END C=17438.16167958451, epsilon=0.09569579080821362, gamma=0.00028875853933915967; total time=   1.3s
[CV] END C=17438.16167958451, epsilon=0.09569579080821362, gamma=0.00028875853933915967; total time=   1.3s
[CV] END C=17438.16167958451, epsilon=0.09569579080821362, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=17438.16167958451, epsilon=0.09569579080821362, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=17438.16167958451, epsilon=0.09569579080821362, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=17438.16167958451, epsilon=0.09569579080821362, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=17438.16167958451, epsilon=0.09569579080821362, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=17438.16167958451, epsilon=0.09569579080821362, gamma=0.007031706915999999; total time=   0.0s
[CV] END C=17438.16167958451, epsilon=0.09569579080821362, gamma=0.007031706915999999; total time=   0.0s
[CV] END C=17438.16167958451, epsilon=0.0956957

[CV] END C=1602.7190592494144, epsilon=0.372720081753474, gamma=0.007031706915999999; total time=   0.0s
[CV] END C=1602.7190592494144, epsilon=0.372720081753474, gamma=0.030645378763400424; total time=   0.0s
[CV] END C=1602.7190592494144, epsilon=0.372720081753474, gamma=0.030645378763400424; total time=   0.0s
[CV] END C=1602.7190592494144, epsilon=0.372720081753474, gamma=0.030645378763400424; total time=   0.0s
[CV] END C=1602.7190592494144, epsilon=0.372720081753474, gamma=0.030645378763400424; total time=   0.0s
[CV] END C=1602.7190592494144, epsilon=0.372720081753474, gamma=0.030645378763400424; total time=   0.0s
[CV] END C=1602.7190592494144, epsilon=0.372720081753474, gamma=0.0031684076500385137; total time=   0.1s
[CV] END C=1602.7190592494144, epsilon=0.372720081753474, gamma=0.0031684076500385137; total time=   0.1s
[CV] END C=1602.7190592494144, epsilon=0.372720081753474, gamma=0.0031684076500385137; total time=   0.1s
[CV] END C=1602.7190592494144, epsilon=0.37272008175

[CV] END C=1602.7190592494144, epsilon=0.24120751171956173, gamma=0.0031684076500385137; total time=   0.1s
[CV] END C=1602.7190592494144, epsilon=0.24120751171956173, gamma=0.0031684076500385137; total time=   0.1s
[CV] END C=1602.7190592494144, epsilon=0.24120751171956173, gamma=0.0031684076500385137; total time=   0.1s
[CV] END C=1602.7190592494144, epsilon=0.24120751171956173, gamma=0.0031684076500385137; total time=   0.1s
[CV] END C=198.91544763490663, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.8354722980853039, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=198.9154476349066

[CV] END C=198.91544763490663, epsilon=0.09569579080821362, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.09569579080821362, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.09569579080821362, gamma=0.00028875853933915967; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.09569579080821362, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.09569579080821362, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.09569579080821362, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.09569579080821362, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.09569579080821362, gamma=0.03939624960877656; total time=   0.0s
[CV] END C=198.91544763490663, epsilon=0.09569579080821362, gamma=0.007031706915999999; total time=   0.0s
[CV] END C=198.91544763490663, epsil

In [7]:
print("Tempo de execução Grid Search: {:.3f} s".format(time_elapsed))
print("Melhores parâmetros: ", grid_search.best_params_)
regressor = SVR(kernel = 'rbf', C = grid_search.best_params_['C'],
                gamma = grid_search.best_params_['gamma'],
                epsilon = grid_search.best_params_['epsilon'])

scores = cross_val_score(regressor, X, y, scoring = 'neg_mean_squared_error', cv=5)
rmse =  np.sqrt(np.mean(np.absolute(scores)))
print("RMSE:", rmse)

Tempo de execução Grid Search: 83.842 s
Melhores parâmetros:  {'C': 1602.7190592494144, 'epsilon': 0.8354722980853039, 'gamma': 0.00028875853933915967}
RMSE: 4.834165296326991


## Bayesian Search

A otimização bayesiana utilizando a bibliotec hyperopt requer a definição de uma função objetivo 
a ser minimizada. Como queremos maximizar a accuracia, utilizamos loss = -acc para ser minimizada. 

In [8]:
def objective_func(search_space):
    C =  search_space['C']
    gamma = search_space['gamma']
    epsilon = search_space['epsilon']
    regressor = SVR(**{'C': 2**C, 'gamma': 2**gamma, 'epsilon': epsilon})
    
    acc = cross_val_score(regressor, X, y, scoring='neg_mean_squared_error', cv=None).mean()
    return {'loss': -acc, 'status': STATUS_OK}

bayesian_search_space = {
    'C': hp.uniform('C', exp_min_C, exp_max_C),
    'gamma': hp.uniform('gamma', exp_min_gamma, exp_max_gamma),
    'epsilon': hp.uniform('epsilon', min_epsilon, max_epsilon)   
}
start = time.time()
best_params = fmin(objective_func, bayesian_search_space, algo=tpe.suggest, max_evals=125)
time_elapsed = time.time() - start
best_parameters = {
    'C': 2**best_params['C'],
    'gamma': 2**best_params['gamma'],
    'epsilon': best_params['epsilon']
}

100%|██████████| 125/125 [01:52<00:00,  1.11trial/s, best loss: 15.153091976316869]


In [9]:
print("Tempo de execução Bayesian Search: {:.3f} s".format(time_elapsed))
print("Melhores parâmetros:", best_parameters)
regressor = SVR(C=best_parameters['C'], gamma=best_parameters['gamma'], epsilon=best_parameters['epsilon'])
scores = cross_val_score(regressor, X, y, scoring = 'neg_mean_squared_error', cv=5)
rmse =  np.sqrt(np.mean(np.absolute(scores)))
print("RMSE:", rmse)

Tempo de execução Bayesian Search: 112.572 s
Melhores parâmetros: {'C': 25339.857447098122, 'gamma': 3.1320582855738275e-05, 'epsilon': 0.5825810101914355}
RMSE: 3.892697262351244


## PSO

In [10]:
def funcao(param):  
    svr = SVR(kernel='rbf', C=param[0], gamma=param[1], epsilon=param[2])
    rmse = make_scorer(mean_squared_error, squared=False, greater_is_better=False)
    scores = cross_val_score(svr, X, y, cv=5, scoring=rmse)#'neg_root_mean_squared_error')
    rmse_mean = -np.mean(scores)
    return rmse_mean

lb = np.array([2**exp_min_C, 2**exp_min_gamma, min_epsilon])
ub = np.array([2**exp_max_C, 2**exp_max_gamma, max_epsilon])

start = time.time()
xopt, fopt = pso(funcao, lb, ub, swarmsize=11, maxiter=11)
time_elapsed = time.time() - start

C_opt = str(xopt[0])
gamma_opt = str(xopt[1])
epsilon_opt = str(xopt[2])
print("Tempo de execução PSO: {:.3f} s".format(time_elapsed))
print("Melhores parâmetros: 'C': {0}, 'gamma': {1}, 'epsilon': {2}".format(C_opt,
                                                                           gamma_opt,
                                                                           epsilon_opt))
print("RMSE: ", str(fopt))

Stopping search: maximum iterations reached --> 11
Tempo de execução PSO: 402.050 s
Melhores parâmetros: 'C': 21817.631831468443, 'gamma': 3.0517578125e-05, 'epsilon': 0.05
RMSE:  3.716425672535194


## Simulated annealing

In [16]:
class SimAnn(Annealer):

    def move(self):
        self.state[0] = 2**np.random.uniform(low = exp_min_C, high = exp_max_C)
        self.state[1] = 2**np.random.uniform(low = exp_min_gamma, high = exp_max_gamma)
        self.state[2] = np.random.uniform(min_epsilon, max_epsilon)
    
    def energy(self):
        svr = SVR(kernel='rbf', C=self.state[0], gamma=self.state[1], epsilon=self.state[2])
        rmse = make_scorer(mean_squared_error, squared=False, greater_is_better=False)
        scores = cross_val_score(svr, X, y, cv=5, scoring=rmse)#'neg_root_mean_squared_error')
        rmse_mean = -np.mean(scores)
        
        return rmse_mean

C = 2**np.random.uniform(low = exp_min_C, high = exp_max_C)
gamma = 2**np.random.uniform(low = exp_min_gamma, high = exp_max_gamma)
epsilon = np.random.uniform(min_epsilon, max_epsilon)
init_state = [C, gamma, epsilon]
sa = SimAnn(init_state)
sa.steps = 125
start = time.time()
best_params, rmse = sa.anneal()
time_elapsed = time.time() - start
print("Tempo de execução Simulated Anneling: {:.3f} s".format(time_elapsed))
print("Melhores parâmetros: 'C': {0}, 'gamma': {1}, 'epsilon': {2}".format(best_params[0],
                                                                           best_params[1],
                                                                           best_params[2]))
print('RMSE: ' + str(rmse))

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          8.13   100.00%     0.00%     0:00:35     0:00:00

Tempo de execução Simulated Anneling: 34.821 s
Melhores parâmetros: 'C': 17644.155559829433, 'gamma': 4.8541623897673706e-05, 'epsilon': 0.8190117619976423
RMSE: 4.304367131249085


## CMA-ES

Semelhantemente ao algoritmo de Simullate Annealing, a implementação do algoritmo de otimização CMA-ES também foi feita via biblioteca optunza. A implementação porém é encapsulada em um sampler default fornecido pela própria biblioteca, assim não sendo necessário a implementação de uma classe sampler específica, somente a função objetivo.

Utilize 125 chamadas da função.

In [17]:
def objective(trial):
    c = trial.suggest_loguniform('c', 2**exp_min_C,  2**exp_max_C)
    gamma = trial.suggest_loguniform('gamma', 2**exp_min_gamma,  2**exp_max_gamma)  
    epsilon = trial.suggest_uniform('epsilon', min_epsilon,  max_epsilon)
    
    svr  = SVR(kernel='rbf', C=c, gamma=gamma, epsilon=epsilon)
    scores = cross_val_score(svr, X, y, scoring = 'neg_mean_squared_error', cv=5)
    rmse =  np.sqrt(np.mean(np.absolute(scores)))   
    return rmse


sampler = optuna.samplers.CmaEsSampler()
cmaes = optuna.create_study(sampler=sampler)
start = time.time()
cmaes.optimize(objective, n_trials=125)
time_elapsed = time.time() - start

[I 2021-05-01 20:26:21,624] A new study created in memory with name: no-name-16e5e443-31c0-490c-b37c-67c237f104b4
[I 2021-05-01 20:26:21,708] Trial 0 finished with value: 8.465055786556485 and parameters: {'c': 12.794736213924208, 'gamma': 0.07309775805876217, 'epsilon': 0.5916476611712732}. Best is trial 0 with value: 8.465055786556485.
[I 2021-05-01 20:26:21,806] Trial 1 finished with value: 6.868697389937358 and parameters: {'c': 44.114493847870385, 'gamma': 0.014175610964580173, 'epsilon': 0.35241554030101996}. Best is trial 1 with value: 6.868697389937358.
[I 2021-05-01 20:26:21,893] Trial 2 finished with value: 6.976418872773422 and parameters: {'c': 40.33646717893502, 'gamma': 0.016154883132687616, 'epsilon': 0.6363769976860011}. Best is trial 1 with value: 6.868697389937358.
[I 2021-05-01 20:26:21,978] Trial 3 finished with value: 6.717883033539625 and parameters: {'c': 25.973602190517266, 'gamma': 0.011784830266778403, 'epsilon': 0.39225288553145254}. Best is trial 3 with valu

[I 2021-05-01 20:26:25,009] Trial 34 finished with value: 5.7752997776536406 and parameters: {'c': 32.394076866551494, 'gamma': 0.0034247340129966116, 'epsilon': 0.606410702822244}. Best is trial 34 with value: 5.7752997776536406.
[I 2021-05-01 20:26:25,099] Trial 35 finished with value: 6.317400052736746 and parameters: {'c': 46.85626143288071, 'gamma': 0.006979148652569083, 'epsilon': 0.7456986587200312}. Best is trial 34 with value: 5.7752997776536406.
[I 2021-05-01 20:26:25,202] Trial 36 finished with value: 6.234929694739111 and parameters: {'c': 31.482314176546343, 'gamma': 0.006418019984690832, 'epsilon': 0.24651388997992307}. Best is trial 34 with value: 5.7752997776536406.
[I 2021-05-01 20:26:25,296] Trial 37 finished with value: 6.021911773620157 and parameters: {'c': 18.10730483086471, 'gamma': 0.003833034135719629, 'epsilon': 0.7385807884733531}. Best is trial 34 with value: 5.7752997776536406.
[I 2021-05-01 20:26:25,396] Trial 38 finished with value: 5.900663609819506 and 

[I 2021-05-01 20:26:28,406] Trial 68 finished with value: 5.416392663960716 and parameters: {'c': 37.20694486242158, 'gamma': 0.0008781301953424637, 'epsilon': 0.3701476649771497}. Best is trial 63 with value: 5.2023923643676335.
[I 2021-05-01 20:26:28,513] Trial 69 finished with value: 5.365026331531703 and parameters: {'c': 72.65124402947801, 'gamma': 0.0018139899784128713, 'epsilon': 0.608991651678102}. Best is trial 63 with value: 5.2023923643676335.
[I 2021-05-01 20:26:28,609] Trial 70 finished with value: 5.410376563252831 and parameters: {'c': 70.52610619571747, 'gamma': 0.0002844895812975066, 'epsilon': 0.46814354569545363}. Best is trial 63 with value: 5.2023923643676335.
[I 2021-05-01 20:26:28,718] Trial 71 finished with value: 5.621170716692155 and parameters: {'c': 69.77939661099838, 'gamma': 0.0028228403799585074, 'epsilon': 0.9663070015974344}. Best is trial 63 with value: 5.2023923643676335.
[I 2021-05-01 20:26:28,921] Trial 72 finished with value: 5.173650229089236 and 

[I 2021-05-01 20:26:33,921] Trial 102 finished with value: 5.4698964566612105 and parameters: {'c': 91.71608033385411, 'gamma': 0.0001228125093975718, 'epsilon': 0.6559451966512442}. Best is trial 88 with value: 4.855245095905023.
[I 2021-05-01 20:26:34,262] Trial 103 finished with value: 5.611148711176918 and parameters: {'c': 842.5633479887487, 'gamma': 0.0009831667928314734, 'epsilon': 0.5341104769405846}. Best is trial 88 with value: 4.855245095905023.
[I 2021-05-01 20:26:34,378] Trial 104 finished with value: 5.066733385862306 and parameters: {'c': 218.1133068412399, 'gamma': 0.00018227543909131737, 'epsilon': 0.9510134734769418}. Best is trial 88 with value: 4.855245095905023.
[I 2021-05-01 20:26:34,517] Trial 105 finished with value: 5.095363518141604 and parameters: {'c': 232.3288558658149, 'gamma': 0.00025103363223884955, 'epsilon': 0.6910334043794064}. Best is trial 88 with value: 4.855245095905023.
[I 2021-05-01 20:26:34,843] Trial 106 finished with value: 4.726377203977423 

In [18]:
print("Tempo de execução CMA-ES: {:.3f} s".format(time_elapsed))
print('Melhores parâmetros:',cmaes.best_params)
print('RMSE:',cmaes.best_value)

Tempo de execução CMA-ES: 18.199 s
Melhores parâmetros: {'c': 2216.197086488184, 'gamma': 6.610965699443206e-05, 'epsilon': 0.3284615721209558}
RMSE: 4.2324142272666485
